<h1>The Battle Of Neighborhoods</h1>
<h2>Problem Desciption</h2>
<b>In Delhi, if someone is looking to open an Indian restaurant, where would you recommend that they open it?</b><br><br>

This project aims to find the best location to open an Indian restaurant in the city of Delhi, India, to maximize the profit of the owner. Our subject wants to own a place at a location that would be best-suited to open an Indian restaurant. Buying property for your Indian restaurant will probably be your most significant start-up expense.  Our subject wants to invest in the best location, so she wants us to analyze all the findings and then recommend some of them. After finding some suitable locations, she will filter them out according to her comforts and budget.

So, we have to provide her the best recommendations, the best locations in Delhi, and she’ll choose them accordingly. 


In [5]:
import requests
import pandas as pd
from sklearn.cluster import KMeans
import json
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
import folium
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.distance import geodesic

<h4>Data Description</h4>
<i>We have been dealing with the following data in our project:-<br>
• Dataset containing  Neighborhoods of Delhi, the dataset holds the names Neighborhood in Delhi extracted from Wikipedia. Here is a link to the dataset.<br>
• I used python library geopy to distill coordinates of neighborhoods in the data frame. Data required some scraping for geopy to correctly give the coordinates of Neighborhoods of Delhi in a data frame.<br>
 • I used Foursquare API to explore the most common places of a neighborhood in the form of a JSON file.</i>

In [6]:
df=pd.read_csv('delhi.csv')
df

,Neighborhood
0,Adarsh Nagar
1,Ashok Vihar
2,Begum Pur
3,Karala
4,Model Town
...,...
112,Rajouri Garden
113,Tihar Village
114,Tilak Nagar
115,Vikas Nagar


<i>Extracting geographical coordinates of given neighborhoods</i>

In [7]:
lat=[]
lon=[]
for x in df.Neighborhood:
    location=None
    
    while location is None:
        geolocator=Nominatim(user_agent="delhi")
        location=geolocator.geocode('{}, Delhi, India'.format(x))
        x=str(x).split(" ")[:-1]
    lat.append(location.latitude)
    print(location)
    print(location.latitude)
    lon.append(location.longitude)
    print(location.longitude)

Adarsh Nagar, Bindapur, Dwarka Tehsil, South West Delhi, Delhi, India
28.6141925
77.0715411848447
Ashok Vihar, Ashok Vihar Road, Ashok Vihar - II, Saraswati Vihar Tehsil, North Delhi, Delhi, 110052, India
28.6994533
77.1848256
Begum Zaidi Market  Moti Bagh - North Facing, F Mohan Senapati Marg, North Moti Bagh, New Delhi, Delhi Cantonment, Delhi, 110021, India
28.584553200000002
77.17884135
Karala, Kanjhawalan Tehsil, North West Delhi, Delhi, India
28.73514
77.0325105
Model Town, Mahatma Gandhi Marg, neighbouhood, Indira Vikas Colony, Model Town Tehsil, North Delhi, Delhi, 110009, India
28.7027136
77.1939912
Narela, Banker, Narela Tehsil, North Delhi, Delhi, 110040, India
28.8426096
77.0918354
Pitam Pura, Outer Ring Road, Rohit Kunj, Saraswati Vihar Tehsil, North West Delhi, Delhi, 110034, India
28.7032676
77.1322497
Rohini, Rohini Tehsil, North West Delhi, Delhi, India
28.7162092
77.1170743
Shalimar Bagh, Saraswati Vihar Tehsil, North West Delhi, Delhi, 110088, India
28.71745265
77.15

<i>Adding coordinates into the dataframe</i>

In [8]:
df["latitude"]=lat
df["longitude"]=lon
df.head()

,Neighborhood,latitude,longitude
0,Adarsh Nagar,28.614193,77.071541
1,Ashok Vihar,28.699453,77.184826
2,Begum Pur,28.584553,77.178841
3,Karala,28.735140,77.032511
4,Model Town,28.702714,77.193991


<i>Visualizing the neighborhoods of Delhi</i>

In [9]:
delhi_map=folium.Map(location=[28.644800,77.216721],zoom_start=10)
for lat, lng, neighborhood in zip(df['latitude'], df['longitude'],df['Neighborhood']):
    label = '{}, Delhi'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(delhi_map)
delhi_map

<i>Foursquare Credentials</i>

In [10]:
CLIENT_ID='HBQ1QJTILHSLNN0PQXFOY0MTA5QYSOCFG0GYKOJYDFBZTYSH'
CLIENT_SECRET='AZXGBWSNOHWWDXWTXKRW5BJV2IGCA1OAUFNA3VNSTXNWRV2G'
LIMIT=100
VERSION='20180604'

<i>Function to get nearby venues of all the neighborhoods</i>

In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<i>Calling the function using dataframe</i>

In [12]:
venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['latitude'],
                                   longitudes=df['longitude']
                                  )

Adarsh Nagar
Ashok Vihar
Begum Pur
Karala
Model Town
Narela
Pitam Pura
Rohini Sub City
Shalimar Bagh
Civil Lines
Gulabi Bagh
Kamla Nagar
Kashmiri Gate
Kotwali
Mori Gate; Delhi
Sadar Bazaar
Sarai Rohilla
Shakti Nagar
Shastri Nagar
Tis Hazari
Timarpur
Wazirabad
Dilshad Garden
Naveen Shahdara
New Usmanpur
Shahdara
Shastri Park
Yamuna Vihar
Chandni Chowk
Daryaganj
Jhandewalan
Karol Bagh
Kishanganj
Paharganj
Rajender Nagar
Barakhamba Road
Chanakyapuri
Connaught Place
Gole Market
INA Colony
Laxmibai Nagar
Pragati Maidan
East Vinod Nagar
Jhilmil Colony
Laxmi Nagar
Mayur Vihar
Pandav Nagar
Preet Vihar
Anand Vihar
Vasundhara Enclave
Vishwas Nagar
Vivek Vihar
Alaknanda
Chittaranjan Park
Defence Colony
East of Kailash
Govindpuri
Greater Kailash
Green Park
Gulmohar Park
Hauz Khas
Hauz Khas Village
Jangpura
Jor Bagh
Kalkaji
Khanpur
Lajpat Nagar
Lodi Colony
Malviya Nagar
Mehrauli
Neeti Bagh
Nehru Place
Netaji Nagar
New Friends Colony
Nizamuddin West
Okhla
Safdarjung Enclave
Sainik Farm
Saket
Sangam 

<i>Dataframe of nearby venues</i>

In [13]:
print(venues.shape)
venues.head()

(819, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adarsh Nagar,28.614193,77.071541,Wordsworth Bookstore,28.611429,77.069148,Bookstore
1,Adarsh Nagar,28.614193,77.071541,Bikanerwala,28.613391,77.076084,Indian Restaurant
2,Ashok Vihar,28.699453,77.184826,SNAP FITNESS 24×7 GYM,28.700180,77.184582,Athletics & Sports
3,Ashok Vihar,28.699453,77.184826,Grand Plaza,28.696577,77.185245,Hotel Bar
4,Ashok Vihar,28.699453,77.184826,Bakers Stop,28.700495,77.188716,Bakery


<i>Count the nearby venues</i>

In [14]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adarsh Nagar,2,2,2,2,2,2
Alaknanda,11,11,11,11,11,11
Anand Vihar,9,9,9,9,9,9
Ashok Nagar,4,4,4,4,4,4
Ashok Vihar,4,4,4,4,4,4
...,...,...,...,...,...,...
Vikaspuri,4,4,4,4,4,4
Vishwas Nagar,4,4,4,4,4,4
Vivek Vihar,1,1,1,1,1,1


<i>Unique Categories</i>

In [15]:
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 164 uniques categories.


<i>Analyzing each category for each neighborhood</i>

In [16]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] =venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

,Neighborhood,ATM,Accessories Store,American Restaurant,Antique Shop,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Adarsh Nagar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Adarsh Nagar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Ashok Vihar,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,Ashok Vihar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Ashok Vihar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<i>Dimensions of given dataframe</i>

In [17]:
onehot.shape

(819, 165)

<i>Grouping by neighborhoods</i>

In [18]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,ATM,Accessories Store,American Restaurant,Antique Shop,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Adarsh Nagar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Alaknanda,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.181818,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Anand Vihar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Ashok Nagar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Ashok Vihar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,Vikaspuri,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
106,Vishwas Nagar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
107,Vivek Vihar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
108,Wazirabad,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<i>Dimensions of given dataframe</i>

In [19]:
grouped.shape

(110, 165)

<i>Best five venues for each neighborhoods</i>

In [20]:
for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head())
    print('\n')

----Adarsh Nagar----
               venue  freq
0  Indian Restaurant   0.5
1          Bookstore   0.5
2                ATM   0.0
3          Nightclub   0.0
4      Moving Target   0.0


----Alaknanda----
               venue  freq
0  Indian Restaurant  0.18
1          BBQ Joint  0.18
2        Pizza Place  0.09
3  Food & Drink Shop  0.09
4         Restaurant  0.09


----Anand Vihar----
                    venue  freq
0       Indian Restaurant  0.22
1             Pizza Place  0.11
2          Ice Cream Shop  0.11
3          Clothing Store  0.11
4  Furniture / Home Store  0.11


----Ashok Nagar----
                  venue  freq
0  Fast Food Restaurant  0.50
1                  Café  0.25
2        Ice Cream Shop  0.25
3                   ATM  0.00
4        Nightlife Spot  0.00


----Ashok Vihar----
                venue  freq
0  Athletics & Sports  0.25
1      Shipping Store  0.25
2           Hotel Bar  0.25
3              Bakery  0.25
4                 ATM  0.00


----Badarpur----
          

<i>Function to return most common venues</i>

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<i>Extracting 10 most common venues in a dataframe</i>

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adarsh Nagar,Indian Restaurant,Bookstore,Yoga Studio,Food Court,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
1,Alaknanda,Indian Restaurant,BBQ Joint,Pizza Place,New American Restaurant,Restaurant,Food & Drink Shop,Coffee Shop,Steakhouse,Middle Eastern Restaurant,Hookah Bar
2,Anand Vihar,Indian Restaurant,Indian Sweet Shop,Furniture / Home Store,Pizza Place,Punjabi Restaurant,Clothing Store,Soup Place,Ice Cream Shop,Hot Dog Joint,Flea Market
3,Ashok Nagar,Fast Food Restaurant,Ice Cream Shop,Café,Yoga Studio,Food Truck,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
4,Ashok Vihar,Bakery,Shipping Store,Hotel Bar,Athletics & Sports,Food & Drink Shop,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
...,...,...,...,...,...,...,...,...,...,...,...
105,Vikaspuri,Indian Restaurant,Flea Market,Pizza Place,American Restaurant,Food Court,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
106,Vishwas Nagar,Electronics Store,Café,Big Box Store,Bank,Gift Shop,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop
107,Vivek Vihar,Juice Bar,Yoga Studio,Food & Drink Shop,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck
108,Wazirabad,Snack Place,Liquor Store,Yoga Studio,Food Court,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck


<i>Using K-means clustering</i>

In [23]:
# set number of clusters
kclusters = 5

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 3, 3, 3, 0, 0, 3, 3])

<i>Adding clustering labels and merge the dataframes</i>

In [33]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

merged = df

# merge grouped with data to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
merged = merged.drop(merged[merged['Cluster Labels'].isna()].index)
merged.reset_index(drop=True,inplace=True)
merged # check the last columns!

,Neighborhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adarsh Nagar,28.614193,77.071541,0.0,Indian Restaurant,Bookstore,Yoga Studio,Food Court,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
1,Ashok Vihar,28.699453,77.184826,3.0,Bakery,Shipping Store,Hotel Bar,Athletics & Sports,Food & Drink Shop,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
2,Begum Pur,28.584553,77.178841,3.0,Convenience Store,Department Store,Museum,Bakery,Yoga Studio,Food Court,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
3,Model Town,28.702714,77.193991,3.0,Pizza Place,Donut Shop,Food Truck,Fast Food Restaurant,Men's Store,Convenience Store,Lake,Snack Place,Bank,Garden
4,Narela,28.842610,77.091835,3.0,Electronics Store,Yoga Studio,Food Court,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,Punjabi Bagh,28.668945,77.132461,3.0,Fast Food Restaurant,Pizza Place,Gym / Fitness Center,Sandwich Place,Garden Center,Breakfast Spot,Hotel,Hot Dog Joint,Frozen Yogurt Shop,Fried Chicken Joint
106,Rajouri Garden,28.642152,77.116060,3.0,Gourmet Shop,BBQ Joint,Gym,Food Truck,Market,Light Rail Station,Cosmetics Shop,Bakery,Indian Restaurant,Train Station
107,Tihar Village,28.634636,77.107112,3.0,Park,Sculpture Garden,Sandwich Place,Yoga Studio,Food & Drink Shop,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck
108,Tilak Nagar,28.639650,77.094039,3.0,Fast Food Restaurant,Electronics Store,Yoga Studio,Food Court,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant


<i>Visualizing all clusters</i>

In [34]:
# create map
map_clusters = folium.Map(location=[28.644800,77.216721], zoom_start=11)
merged.dropna()
merged.reset_index(drop=True,inplace=True)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['latitude'], merged['longitude'], merged['Neighborhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h4>Cluster 0</h4>

In [49]:
merged.loc[merged['Cluster Labels'] == 0,merged.columns[[0] + list(range(4, merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adarsh Nagar,Indian Restaurant,Bookstore,Yoga Studio,Food Court,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
5,Pitam Pura,Women's Store,Indian Restaurant,Gym,Market,Food Truck,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
7,Civil Lines,Italian Restaurant,Asian Restaurant,Café,Chinese Restaurant,Hotel,Burger Joint,Indian Restaurant,Fish & Chips Shop,Fried Chicken Joint,Garden Center
9,Kamla Nagar,Breakfast Spot,Indian Restaurant,Coffee Shop,Pizza Place,Miscellaneous Shop,Donut Shop,Flea Market,Chinese Restaurant,Fried Chicken Joint,Parking
11,Kotwali,Indian Restaurant,Hotel,Hostel,Soccer Stadium,Cricket Ground,Asian Restaurant,History Museum,Restaurant,Road,Food
20,Dilshad Garden,Pizza Place,Diner,Light Rail Station,Indian Restaurant,Antique Shop,Food Truck,Gastropub,Garden Center,Garden,Furniture / Home Store
26,Chandni Chowk,Indian Restaurant,Snack Place,Market,Paper / Office Supplies Store,Fast Food Restaurant,Flea Market,Hotel,Dessert Shop,Falafel Restaurant,Farmers Market
27,Daryaganj,Indian Restaurant,Restaurant,Hotel,Road,Falafel Restaurant,Food & Drink Shop,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
32,Barakhamba Road,Theater,Indian Restaurant,Art Gallery,Bakery,Café,Concert Hall,Hotel Bar,Arcade,Light Rail Station,Gastropub
34,Connaught Place,Indian Restaurant,Chinese Restaurant,Café,Bar,Lounge,North Indian Restaurant,Coffee Shop,Pub,Italian Restaurant,Deli / Bodega


<h4>Cluster 1</h4>

In [36]:
merged.loc[merged['Cluster Labels'] == 1,merged.columns[[0] + list(range(4, merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
40,Jhilmil Colony,Juice Bar,Asian Restaurant,Bed & Breakfast,Yoga Studio,Food Court,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
48,Vivek Vihar,Juice Bar,Yoga Studio,Food & Drink Shop,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck
87,Inderpuri,Stables,Gym,Juice Bar,Yoga Studio,Food Court,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant


<h4>Cluster 2</h4>

In [37]:
merged.loc[merged['Cluster Labels'] == 2,merged.columns[[0] + list(range(4, merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
60,Jor Bagh,ATM,Gift Shop,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court
91,Najafgarh,ATM,Food & Drink Shop,Bank,Food Truck,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint


<h4>Cluster 3</h4>

In [38]:
merged.loc[merged['Cluster Labels'] == 3,merged.columns[[0] + list(range(4, merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Ashok Vihar,Bakery,Shipping Store,Hotel Bar,Athletics & Sports,Food & Drink Shop,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
2,Begum Pur,Convenience Store,Department Store,Museum,Bakery,Yoga Studio,Food Court,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
3,Model Town,Pizza Place,Donut Shop,Food Truck,Fast Food Restaurant,Men's Store,Convenience Store,Lake,Snack Place,Bank,Garden
4,Narela,Electronics Store,Yoga Studio,Food Court,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck
6,Rohini Sub City,Shopping Mall,Department Store,Indian Restaurant,Fast Food Restaurant,Café,Gym,Light Rail Station,Fish & Chips Shop,Flea Market,Food
...,...,...,...,...,...,...,...,...,...,...,...
104,Patel Nagar,Café,Train Station,Yoga Studio,Food & Drink Shop,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck
105,Punjabi Bagh,Fast Food Restaurant,Pizza Place,Gym / Fitness Center,Sandwich Place,Garden Center,Breakfast Spot,Hotel,Hot Dog Joint,Frozen Yogurt Shop,Fried Chicken Joint
106,Rajouri Garden,Gourmet Shop,BBQ Joint,Gym,Food Truck,Market,Light Rail Station,Cosmetics Shop,Bakery,Indian Restaurant,Train Station
107,Tihar Village,Park,Sculpture Garden,Sandwich Place,Yoga Studio,Food & Drink Shop,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck


<h4>Cluster 4</h4>

In [39]:
merged.loc[merged['Cluster Labels'] == 4,merged.columns[[0] + list(range(4, merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Shastri Nagar,Light Rail Station,Shopping Mall,Flea Market,Yoga Studio,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck
30,Kishanganj,Light Rail Station,Train Station,Yoga Studio,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck
89,Moti Bagh,Light Rail Station,Shopping Mall,Yoga Studio,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court


<i>By observation at each cluster’s dataframe, it is very clear that Cluster 0 contains less restaurants but comprises of areas with great footfall. So, to open an Indian restaurant, Cluster 0 is the best match.<br> 
I eliminated the neighborhoods which have any restaurant as their near by venue. So, there are total 24 neighborhoods.</i>

In [51]:
best_venues=merged.loc[merged['Cluster Labels'] == 3]
for col in best_venues.columns:
    if(col!="latitude" and col!="longitude" and col!="Cluster Labels"):
        best_venues = best_venues.drop(best_venues[best_venues[col].str.contains('Restaurant', na = False)].index)
best_venues.reset_index(drop=True,inplace=True)
best_venues

,Neighborhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Begum Pur,28.584553,77.178841,3.0,Convenience Store,Department Store,Museum,Bakery,Yoga Studio,Food Court,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
1,Pragati Maidan,28.623475,77.242528,3.0,Plaza,Art Gallery,Hotel,Light Rail Station,Yoga Studio,Food & Drink Shop,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
2,Laxmi Nagar,28.630553,77.277575,3.0,Shopping Mall,Light Rail Station,Accessories Store,Mobile Phone Shop,Train Station,Gym,Snack Place,Garden,Furniture / Home Store,Frozen Yogurt Shop
3,Vishwas Nagar,28.664436,77.294960,3.0,Electronics Store,Café,Big Box Store,Bank,Gift Shop,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop
4,Chittaranjan Park,28.538752,77.249249,3.0,Market,Karaoke Bar,Hookah Bar,Yoga Studio,Food Court,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
5,Govindpuri,28.535156,77.263794,3.0,Electronics Store,Clothing Store,Farmers Market,Bar,Burger Joint,Gift Shop,Gastropub,Garden Center,Garden,Furniture / Home Store
6,Sarita Vihar,28.528574,77.288331,3.0,Park,Train Station,Juice Bar,Market,Yoga Studio,Food Court,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
7,Munirka,28.554886,77.171084,3.0,Pharmacy,Bank,Grocery Store,Furniture / Home Store,Flea Market,Department Store,Pizza Place,Playground,Sandwich Place,Spa
8,Kirti Nagar,28.653281,77.141773,3.0,Pizza Place,BBQ Joint,Fish & Chips Shop,Women's Store,Antique Shop,Food Truck,Gastropub,Garden Center,Garden,Furniture / Home Store
9,Paschim Vihar,28.669578,77.095956,3.0,Garden,Jewelry Store,BBQ Joint,Coffee Shop,Yoga Studio,Food Court,Garden Center,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint


In [57]:
delhi_map=folium.Map(location=[28.644800,77.216721],zoom_start=11)
for lat, lng, neighborhood in zip(best_venues['latitude'], best_venues['longitude'],best_venues['Neighborhood']):
    label = '{}, Delhi'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(delhi_map)
delhi_map